# Exercise 04. (Take-home) Advanced Topic: Introduction to Offline RL and Serving your RLlib Model using Ray Serve API

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives
In this this tutorial, you will learn:
 * [What's offline RL (aka "batch RL")?](#offline_rl)
 * [How to configure RLlib for offline RL](#offline_rl_with_rllib)

## What's offline RL (aka "batch RL")? <a class="anchor" id="offline_rl"></a>

So far, we have dealt with a so-called "online" setting for RL: We had direct influence over a live-running environment (a simulator). We were able to send arbitrary actions to this simulator and collect its responses (rewards and observations), thereby learning "as we go". This setup is called "online" RL:

<img src="images/online_rl.png" width="80%"></img>

However, often and especially in real-life industry settings, we are faced with the problem of not having a simulator at hand.
In this case, we need to fall back to offline RL:

<img src="images/offline_rl.png" width="70%"></img>


**Note:** Due to the dynamic nature of adversarial multi-agent scenarios, we will cover the topic of
of offline RL here only for the single-agent case.
Research on multi-agent offline RL is bleeding edge and not well explored by RLlib thus far (see references).

Offline RL comes in two flavours:
* Pure imitation learning (the agent will try to imitate 100% the actions/behavior that it finds in the offline data). This setup is nothing else but supervised learning with a `-log(p)` loss function.
* Imitation learning plus improvement over the recorded, offline behavior.

In fact, modern offline RL algorithms are capable of performing imitation plus improvement, such that they can learn to perfectly play e.g. the CartPole environment, when only behavioral data from a randomly acting agent is available! We'll explore this right now using RLlib's new CRR algorithm.

In [10]:
# Learning a decent policy using offline RL requires specialized RL algorithms.
# Examples of offline RL algos are RLlib's "CRR", "MARWIL", or "CQL".
# For this example, we'll use the "Pendulum-v0" environment and have the "CRR"
# (critic regularized regression) algorithm learn how to solve this environment, purely from
# data recorded from a random/beginner agent.


# Import the config class of the algorithm, we would like to train with: CRR.
from ray.rllib.algorithms.crr import CRRConfig

# Create a defaut CRR config:
config = CRRConfig()

# Set it up for the correct environment:
# NOTE: We said above that we wouldn't really have an environment available (so how can
# we set one up here??).
# The following is only to tell the algorithm
config.environment(env="Pendulum-v1")

#################################################
# This is the most important piece of code 
# in this notebook:
# It explains how to point your 
# algorithm to the correct offline data file
# (instead of a live-environment).
#################################################
config.offline_data(
    input_="dataset",
    input_config={
        "paths": "offline_rl_data/pendulum.zip",
        "format": "json",
    },
    actions_in_input_normalized=True,
)

# RLlib's CRR is a very new algorithm (since 1.13) and only supports
# the PyTorch framework thus far. We'll provide a tf version in the near future.
config.framework("torch")

# Set up evaluation as follows:
config.evaluation(
    # Run evaluation once per `train()` call (by default, RLlib will evaluate 10 episodes).
    evaluation_interval=1,
    # Use a separate resource ("RLlib rollout worker")
    evaluation_num_workers=1,
    # Run evaluation parallel to training.
    evaluation_parallel_to_training=True,
    # Use a slightly different config for the evaluation:
    evaluation_config={
        # - Use a real environment (so we can fully trust the evaluation results, rewards, etc..)
        "input": "sampler",
        # - Switch off exploration for better (less stochastic) action computations.
        "explore": False,
    },
)


### Exercises

1. Keep configuring our CRR algorithm by calling the config object's `training()` method and passing the following settings into that call:

```
gamma: 0.99
train_batch_size: 1024
critic_lr: 0.0003
actor_lr: 0.0003
target_network_update_freq: 1
tau: 0.0001
weight_type: "exp"
```

In [7]:
# Make the `training()` call on your config here in this cell:
config.training()  # <- fill this out

2. Use `tune.run()` to kick off the experiment, similar to how we did it in the previous notebook. Let's see how fast CRR can learn to play pendulum from scratch (from beginner's data)! As stopping criteria, use `timesteps_total=2000000` and `evaluation/episode_reward_mean=-300`.

In [9]:
# Perform the `tune.run()` call here:

from ray import tune

tune.run(
    "CRR",
    # config=...  <- check out the previous notebook on how to use tune.run() with an RLlib config object
    # ...
)

2022-07-24 17:27:37,953	INFO services.py:1477 -- View the Ray dashboard at http://127.0.0.1:8268
2022-07-24 17:27:41,123	WARNING deprecation.py:47 -- DeprecationWarning: `ray.rllib.execution.buffers` has been deprecated. Use `ray.rllib.utils.replay_buffers` instead. This will raise an error in the future!
2022-07-24 17:27:41,517	INFO plugin_schema_manager.py:51 -- Loading the default runtime env schemas: ['/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.9/site-packages/ray/_private/runtime_env/../../runtime_env/schemas/working_dir_schema.json', '/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.9/site-packages/ray/_private/runtime_env/../../runtime_env/schemas/pip_schema.json'].
2022-07-24 17:27:51,606	ERROR trial_runner.py:920 -- Trial CRR_282d2_00000: Error processing event.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.9/site-packages/ray/tune/execution/ray_trial_ex

== Status ==
Current time: 2022-07-24 17:27:51 (running for 00:00:10.36)
Memory usage on this node: 11.9/16.0 GiB
Using FIFO scheduling algorithm.
Resources requested: 5.0/16 CPUs, 0/0 GPUs, 0.0/4.79 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/sven/ray_results/CRR
Number of trials: 1/1 (1 RUNNING)
+-----------------+----------+-------+
| Trial name      | status   | loc   |
|-----------------+----------+-------|
| CRR_282d2_00000 | RUNNING  |       |
+-----------------+----------+-------+


Result for CRR_282d2_00000:
  trial_id: 282d2_00000
  
== Status ==
Current time: 2022-07-24 17:27:51 (running for 00:00:10.38)
Memory usage on this node: 11.9/16.0 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/16 CPUs, 0/0 GPUs, 0.0/4.79 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/sven/ray_results/CRR
Number of trials: 1/1 (1 ERROR)
+-----------------+----------+-------+
| Trial name      | status   | loc   |
|-----------------+----------+-------|
| CRR_282d2_00000 | E

TuneError: ('Trials did not complete', [CRR_282d2_00000])

### References

* 